# BEECN Location Visualization

In [1]:
import folium
m = folium.Map(location=[45.5236, -122.6750])

In [2]:
import geopandas as gpd
url_neighborhoods = 'https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson'
neighborhood_geo = gpd.read_file(url_neighborhoods)

folium.Choropleth(
    geo_data=neighborhood_geo,
    fill_color='YlGn',
    fill_opacity=0.1
).add_to(m)

In [3]:
import pandas as pd
import numpy as np
url_beecns = 'https://opendata.arcgis.com/datasets/6e6185533d5447deb8b7204c27e1858e_92.geojson'
beecn_geo = gpd.read_file(url_beecns)
beecn_geo = pd.DataFrame(beecn_geo)

beecn_geo['geometry'] = beecn_geo['geometry'].map(lambda x: str(x).lstrip('POINT (').rstrip(')'))
lat = []
lon = []

for row in beecn_geo['geometry']:
    try:
        lon.append(row.split(' ')[0])
        lat.append(row.split(' ')[1])
    except:
        lon.append(np.NaN)
        lat.append(np.NaN)

beecn_geo['latitude'] = lat
beecn_geo['longitude'] = lon
print(beecn_geo.head())

   OBJECTID                          SITE_NAME  \
0         1         Parkrose High School Field   
1         2  Ainsworth Elementary School Field   
2         3    Chapman Elementary School Field   
3         4   Sunstone Montessori School Field   
4         5                     Lair Hill Park   

                              LOCATION SITE_OWNER    X_COORD   Y_COORD MAP_ID  \
0               12003 NE Shaver Street        PPS  7678990.0  694114.0  NE-19   
1     SW Elm Street and SW 21st Avenue        PPS  7638110.0  679716.0  SW-24   
2  NW Quimby Street and NW 27th Avenue        PPS  7636770.0  688215.0   NW-3   
3               6318 SW Corbett Street        PPS  7643970.0  667980.0  SW-29   
4    SW 2nd Aveue and SW Porter Street       PP&R  7643160.0  676494.0  SW-25   

  BARRIER_ID                              geometry           latitude  \
0          L  -122.5422587229825 45.55243515646136  45.55243515646136   
1          F  -122.7002495885055 45.51001783951922  45.51001783951

## Generate a Map

Generate a map of the BEECN Locations with a 1600 meter radius around each site. The map also shows the boundaries of each Portland Neighborhood. 

In [9]:
latitude = np.array(beecn_geo['latitude'])
latitude = latitude.astype(float)
longitude = np.array(beecn_geo['longitude'])
longitude = longitude.astype(float)

beecn_site = beecn_geo['SITE_NAME']
address = beecn_geo['LOCATION']

location = zip(latitude, longitude)
count = 0
for i in location:
    print(i)
    folium.Marker(i, popup=address[count], icon= folium.Icon(icon='medkit', prefix='fa'), tooltip=beecn_site[count]).add_to(m)
    folium.Circle(i, radius=1600).add_to(m)
    count += 1
m

(45.55243515646136, -122.5422587229825)
(45.51001783951922, -122.7002495885055)
(45.53321840593239, -122.7063696524326)
(45.47827510394902, -122.6761713953615)
(45.50155787224017, -122.6802225170186)
(45.44137550298379, -122.7031286548551)
(45.45447172901483, -122.6852775967354)
(45.49113360293868, -122.7226774587931)
(45.45516833899535, -122.7258019915907)
(45.44716215847855, -122.7262261428302)
(45.59276082697463, -122.7517854773162)
(45.57509216109042, -122.7301261462969)
(45.46797290986859, -122.6284821141569)
(45.48317838922451, -122.6114492344115)
(45.50890464743873, -122.5757870599647)
(45.52836270492966, -122.5782524898552)
(45.56582656641949, -122.6452468791293)
(45.57218939646528, -122.6542424511793)
(45.55624138923265, -122.6090991751529)
(45.5431869093847, -122.588166186838)
(45.54980556857069, -122.5044943946511)
(45.45932519093659, -122.5434315526377)
(45.47237478216439, -122.5062714751839)
(45.46801351180761, -122.5859762370074)
(45.48966726599637, -122.5699794273983)
(4

<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVM9ZmFsc2U7IExfTk9fVE9VQ0g9ZmFsc2U7IExfRElTQUJMRV8zRD1mYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS40LjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2NvZGUuanF1ZXJ5LmNvbS9qcXVlcnktMS4xMi40Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS40LjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdjZG4uZ2l0aGFjay5jb20vcHl0aG9uLXZpc3VhbGl6YXRpb24vZm9saXVtL21hc3Rlci9mb2xpdW0vdGVtcGxhdGVzL2xlYWZsZXQuYXdlc29tZS5yb3RhdGUuY3NzIi8+CiAgICA8c3R5bGU+aHRtbCwgYm9keSB7d2lkdGg6IDEwMCU7aGVpZ2h0OiAxMDAlO21hcmdpbjogMDtwYWRkaW5nOiAwO308L3N0eWxlPgogICAgPHN0eWxlPiNtYXAge3Bvc2l0aW9uOmFic29sdXRlO3RvcDowO2JvdHRvbTowO3JpZ2h0OjA7bGVmdDowO308L3N0eWxlPgogICAgCiAgICA8bWV0YSBuYW1lPSJ2aWV3cG9ydCIgY29udGVudD0id2lkdGg9ZGV2aWNlLXdpZHRoLAogICAgICAgIGluaXRpYWwtc2NhbGU9MS4wLCBtYXhpbXVtLXNjYWxlPTEuMCwgdXNlci1zY2FsYWJsZT1ubyIgLz4KICAgIDxzdHlsZT4jbWFwXzM5N2IyYTVhMWIxYzRiNjk4NGVmN2YyNDhjZjE4OTRjIHsKICAgICAgICBwb3NpdGlvbjogcmVsYXRpdmU7CiAgICAgICAgd2lkdGg6IDEwMC4wJTsKICAgICAgICBoZWlnaHQ6IDEwMC4wJTsKICAgICAgICBsZWZ0OiAwLjAlOwogICAgICAgIHRvcDogMC4wJTsKICAgICAgICB9CiAgICA8L3N0eWxlPgo8L2hlYWQ+Cjxib2R5PiAgICAKICAgIAogICAgPGRpdiBjbGFzcz0iZm9saXVtLW1hcCIgaWQ9Im1hcF8zOTdiMmE1YTFiMWM0YjY5ODRlZjdmMjQ4Y2YxODk0YyIgPjwvZGl2Pgo8L2JvZHk+CjxzY3JpcHQ+ICAgIAogICAgCiAgICAKICAgICAgICB2YXIgYm91bmRzID0gbnVsbDsKICAgIAoKICAgIHZhciBtYXBfMzk3YjJhNWExYjFjNGI2OTg0ZWY3ZjI0OGNmMTg5NGMgPSBMLm1hcCgKICAgICAgICAnbWFwXzM5N2IyYTVhMWIxYzRiNjk4NGVmN2YyNDhjZjE4OTRjJywgewogICAgICAgIGNlbnRlcjogWzQ1LjUyMzYsIC0xMjIuNjc1XSwKICAgICAgICB6b29tOiAxMCwKICAgICAgICBtYXhCb3VuZHM6IGJvdW5kcywKICAgICAgICBsYXllcnM6IFtdLAogICAgICAgIHdvcmxkQ29weUp1bXA6IGZhbHNlLAogICAgICAgIGNyczogTC5DUlMuRVBTRzM4NTcsCiAgICAgICAgem9vbUNvbnRyb2w6IHRydWUsCiAgICAgICAgfSk7CgoKICAgIAogICAgdmFyIHRpbGVfbGF5ZXJfMjJlYmNhN2FhYjY4NGE4OThmYTQ3NWY0MDFjY2VkYmYgPSBMLnRpbGVMYXllcigKICAgICAgICAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLAogICAgICAgIHsKICAgICAgICAiYXR0cmlidXRpb24iOiBudWxsLAogICAgICAgICJkZXRlY3RSZXRpbmEiOiBmYWxzZSwKICAgICAgICAibWF4TmF0aXZlWm9vbSI6IDE4LAogICAgICAgICJtYXhab29tIjogMTgsCiAgICAgICAgIm1pblpvb20iOiAwLAogICAgICAgICJub1dyYXAiOiBmYWxzZSwKICAgICAgICAib3BhY2l0eSI6IDEsCiAgICAgICAgInN1YmRvbWFpbnMiOiAiYWJjIiwKICAgICAgICAidG1zIjogZmFsc2UKfSkuYWRkVG8obWFwXzM5N2IyYTVhMWIxYzRiNjk4NGVmN2YyNDhjZjE4OTRjKTsKICAgIAogICAgICAgICAgICB2YXIgY2hvcm9wbGV0aF9jYjMxMTQyNzBlNWQ0MGNiYTU2M2MxZmJjYjM0ZjJkNiA9IEwuZmVhdHVyZUdyb3VwKAogICAgICAgICAgICAgICAgKS5hZGRUbyhtYXBfMzk3YjJhNWExYjFjNGI2OTg0ZWY3ZjI0OGNmMTg5NGMpOwogICAgICAgIAogICAgCiAgICAgICAgdmFyIGdlb19qc29uXzFkZjRhMzJmZGI3YTRjNWM5ZGE3MzJmOWY1YTJmNGY3ID0gTC5nZW9Kc29uKAogICAgICAgICAgICB7ImJib3giOiBbLTEyMi44Njc2NTQzMTA